## Navigators
Navigators provide the ability to drill into the result set providing a summary of the distribution.  They are particularly useful when you are interested in gathering the domain of values for a particular Property.  Navigators can be used against a specific View, used in conjunction with either a query, a filter or both.  Navigators can be simple or very powerful, but provide a very useful way to capture results in logical buckets. 

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.Data.Content, 1.0.0-beta4"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.Data.Analysis, 0.20.1 Refinitiv.Data.Content, 1.0.0-beta4

Loading extensions from `C:\Users\U8007876\.nuget\packages\microsoft.data.analysis\0.20.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.Data.Content.SearchService;
using Refinitiv.Data.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Data;
using System;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<DataTable>((dt, writer) =>
{
    if (dt.Columns.Count > 0)
    {
        var rows = new List<ICollection<IHtmlContent>>();

        // Process Each data row
        foreach(var row in dt.Rows.Cast<DataRow>())
        {
            var cells = new List<IHtmlContent>();
            foreach (var cell in row.ItemArray)
            {
                switch (cell)
                {
                    case null:
                        break;
                    case JValue val:
                        cells.Add(td(val.ToString()));
                        break;
                    case IEnumerable<JToken> list:
                        cells.Add(td($"[{string.Join(", ", list.Select(node => $"{node}"))}]"));
                        break;
                    default:
                        cells.Add(td(cell));
                        break;
                }                
            }
            rows.Add(cells);
        }
        
        // Header
        var headers = new List<IHtmlContent>();
        foreach (DataColumn col in dt.Columns)
            headers.Add(th(col.ColumnName) as IHtmlContent);

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
DesktopSession.Definition().GetSession().OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                                        .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})"))
                                        .Open();

2023-02-21 4:03:02 PM:DesktopSession. (State: DesktopSession is Pending)
2023-02-21 4:03:04 PM:DesktopSession. (Event: {
  "Contents": "Desktop Session Successfully Authenticated"
})
2023-02-21 4:03:04 PM:DesktopSession. (State: DesktopSession is Opened)


#### Example - Property values
The most basic usage of navigators is to determine the available values within a given Property.  For example, when searching for assets, we may want to narrow down the specific country an asset is traded in.  We can determine the available countries to work with.

In [5]:
// The following will list the countries assets are listed.  We can use this output to filter based on a specific country.
var response = Search.Definition(Search.View.SearchAll).Top(0)
                                                       .Navigators("RCSCountryLeaf(buckets:20)")  // Show the top 20
                                                       .GetData();
Console.WriteLine(response.Data.Navigators["RCSCountryLeaf"]["Buckets"]);

[
  {
    "Label": "United States",
    "Count": 13669059
  },
  {
    "Label": "Eurobond",
    "Count": 4560764
  },
  {
    "Label": "China (Mainland)",
    "Count": 1289076
  },
  {
    "Label": "Colombia",
    "Count": 1093363
  },
  {
    "Label": "South Korea",
    "Count": 583887
  },
  {
    "Label": "Germany",
    "Count": 506187
  },
  {
    "Label": "Canada",
    "Count": 496157
  },
  {
    "Label": "France",
    "Count": 273193
  },
  {
    "Label": "India",
    "Count": 212292
  },
  {
    "Label": "Japan",
    "Count": 165382
  },
  {
    "Label": "Italy",
    "Count": 135286
  },
  {
    "Label": "Australia",
    "Count": 134035
  },
  {
    "Label": "Chile",
    "Count": 131629
  },
  {
    "Label": "Brazil",
    "Count": 82661
  },
  {
    "Label": "Switzerland",
    "Count": 72073
  },
  {
    "Label": "Austria",
    "Count": 60904
  },
  {
    "Label": "United Kingdom",
    "Count": 60274
  },
  {
    "Label": "Spain",
    "Count": 59978
  },
  {
    "Label": "Norwa

#### Example - Vessels
If I want to perform a basic search for tankers and capture the Hull Types, I could start with the following basic query:

In [6]:
response = Search.Definition(Search.View.VesselPhysicalAssets).Query("tanker")
                                                              .Select("DocumentTitle, HullType")
                                                              .Top(100)
                                                              .GetData();
response.Data.Table

"M.T.THEA, Chemical Tanker, FAIRDEAL GROUP MANAGEMENT, English Channel|UK Continent Tanker Zone",Single Hull
"KK MARLIN, Chemical Tanker, South Africa|Western Cape|South Atlantic Ocean|South Africa Tanker Zone",
"HAFNIA NANJING, Jet Fuel; Chemical Tanker, MMS CO LTD, East Atlantic Ocean (Africa)|Gibraltar Tanker Zone",
"GW DOLPHIN, Chemical Tanker, English Channel|UK Continent Tanker Zone",
"MARITIME TRANQUILITY, Chemical Tanker, Caribbean Sea|Bahamas Tanker Zone|Dominican Republic",
"MARITIME AMITY, Chemical Tanker, East Med Sea|East Med Tanker Zone",
"MARITIME COMITY, Chemical Tanker, Gulf of Mexico|US Gulf Tanker Zone|Texas|United States",
"MARITIME VALOR, Chemical Tanker, East Sea / Sea of Japan|Korea Tanker Zone|Korea Strait Waypoint",
"MARITIME GRACIOUS, Chemical Tanker, MSI SHIP MANAGEMENT PTE LTD, East China Sea|Korea Tanker Zone|Korea Strait Waypoint",
"MARITIME GUARDIAN, Chemical Tanker, MSI SHIP MANAGEMENT PTE LTD, Pacific Ocean (Central America)|Hawaii Tanker Zone",
"MARITIME INSPIRATION, Soybean Oil; Chemical Tanker, Bay of Bengal|Chittagong Tanker Zone",


Here, we can see a list of the different tankers and their Hull Types. Within this collection, while I can clearly see there are 2 different Hull Types: "Single" and "Double", I'm also seeing duplicates.  Instead of going through the effort to process the above result to pull out the unique list of Hull Types, I will instead use a Navigator - which was designed to do this work for me.

Using the above query, I will apply a Navigator by selecting the 'HullType' Property.  What this does is instruct search to bucket all unique Hull Types and summarize the total found for each. To capture the results, I will need to utilize the Content Layer API call.

In [7]:
response = Search.Definition(Search.View.VesselPhysicalAssets).Query("tanker")
                                                              .Top(0)  // Only interested in Navigator details
                                                              .Navigators("HullType")
                                                              .GetData();
Console.WriteLine(response.Data.Navigators["HullType"]["Buckets"]);

[
  {
    "Label": "Double Hull",
    "Count": 16467
  },
  {
    "Label": "Single Hull",
    "Count": 5411
  }
]


#### Example - Listing of exchanges per asset category
In the following example, I'm going to list the top 3 asset types in Canada and for each, display the top 2 exchanges trading on those assets.

In [8]:
// Filter on all active (AC) assets within Canada
var filter = "AssetState eq 'AC' and RCSExchangeCountryLeaf eq 'Canada'";

// For the top 3 categories, show the top 2 exchanges
var navigators = "RCSAssetCategoryLeaf(buckets:3, sub:ExchangeName(buckets:2))"; 

response = Search.Definition(Search.View.EquityQuotes).Filter(filter)
                                                      .Navigators(navigators)
                                                      .Top(0)
                                                      .GetData();

In [9]:
// Pretty display of the listing
var rank = 1;
var exchangeRank = 1;
foreach (var asset in response.Data.Navigators["RCSAssetCategoryLeaf"]["Buckets"])
{
    Console.WriteLine($"{rank++}. {asset["Label"]}");
    foreach (var exchange in asset["ExchangeName"]["Buckets"])
        Console.WriteLine($"\t{exchangeRank++}. {exchange["Label"]} ({exchange["Count"]})");
    exchangeRank = 1;
}

1. Equity Cash Option
	1. Montreal Options Exchange (93129)
	2. Refinitiv (10679)
2. Ordinary Share
	1. Aequitas NEO-L (3569)
	2. Aequitas NEO-N (3568)
3. Equity Future Spread
	1. Montreal Exchange (6432)
	2. Montreal Options Exchange (36)


#### Example - Outstanding Government debt in the last 3 months.
In this example, I will be using some nested expressions to summarize categories within the buckets I collect.  Specifically, I'm looking for government debt ('govt') issued any time after 3-months ago using the following filter:
- "DBType eq 'govt' and IssueDate gt today-3months" (DBType identifies government debt)
    
Based on the result set, I want categorize or create buckets (top 5) based on the domicile of the issuer.  For each domicile, sum the outstanding debt and also include the maximum coupon rate for each one.
- RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)

In [10]:
// Get date N months ago
var date = DateTime.Now.AddMonths(-3);  // Last 3 months

In [11]:
// Retrieve the top 5 outstanding government debt in the last 3 months
filter = $"DbType eq 'govt' and IssueDate gt {date:yyyy-MM-dd}";
navigators = "RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)";

response = Search.Definition(Search.View.GovCorpInstruments).Filter(filter)
                                                            .Navigators(navigators)
                                                            .Top(0)
                                                            .GetData();
Console.WriteLine(response.Data.Navigators["RCSDomicileLeaf"]["Buckets"]);

[
  {
    "Label": "United States",
    "Count": 83,
    "max_CouponRate": 4.9360272,
    "sum_FaceOutstandingUSD": 2338989202900.0
  },
  {
    "Label": "Japan",
    "Count": 51,
    "max_CouponRate": 1.6,
    "sum_FaceOutstandingUSD": 902047497263.0
  },
  {
    "Label": "China (Mainland)",
    "Count": 36,
    "max_CouponRate": 2.8,
    "sum_FaceOutstandingUSD": 352815879183.0
  },
  {
    "Label": "Singapore",
    "Count": 42,
    "max_CouponRate": 4.4944,
    "sum_FaceOutstandingUSD": 222540043497.0
  },
  {
    "Label": "Hong Kong",
    "Count": 56,
    "max_CouponRate": 5.25,
    "sum_FaceOutstandingUSD": 136167813408.0
  }
]
